## Clean the raw data

Load Packages

In [1]:
import pandas as pd
import numpy as np

Read File

In [2]:
path = 'raw.csv'
Russell_data = pd.read_csv(path)

Get rid of row which contains average and delete the "Versus" column

In [3]:
Russell_data = Russell_data[Russell_data['Date'] != "Average"]

Russell_data = Russell_data.drop("Versus", axis=1)

I assueme the 3P% and FT% to be zero if they didn't make any 3PA or FTA

In [4]:
Russell_data['3P%'].fillna(0, inplace=True)
Russell_data['FT%'].fillna(0, inplace=True)

Drop invalid value

In [5]:
Russell_data = Russell_data.dropna()   # Actually, there is no na value.

Split the score to three parts (Win/Lose; Team score; Opponent score)

In [6]:
Russell_data['Win/Lose'] = Russell_data['Score'].str[0]

score_split = Russell_data['Score'].str[1:].str.split('-', expand=True)

Russell_data['Team score'] = score_split[0]

Russell_data['Opponent score'] = score_split[1]

# Drop the original column of score
Russell_data = Russell_data.drop("Score", axis=1)


Split Date

In [7]:
Russell_data['DayOfWeek'] = Russell_data['Date'].str[:3]  # First three letters
Russell_data['date'] = Russell_data['Date'].str[3:]  # Rest of the string
Russell_data = Russell_data.drop("Date", axis=1)

Create new CSV file


In [8]:
Russell_data.to_csv('Reorgnized_RB.csv', index=False)